In [14]:
## In-Class Assignment 16 - Evan Callaghan

import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## 1. Using the pandas library to read the csv data file and create a data-frame called heart
heart = pd.read_csv(file_content_stream)

heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [15]:
## 2. Removing observations with missing values

heart = heart.dropna()

In [16]:
## 3. Using male, age, currentSmoker, totChol, sysBP,BMI, heartRate, and glucose as the predictor variables and TenYearCHD 
## as the target variable to split the data into two data-frames (taking into account the proportion of 0s and 1s): 
## train (80%) and test (20%)

X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [17]:
## 4. Using the train data-frame to build an AdaBoost model (with 500 trees, max tree depth equal
## 3, and learning rate equal to 0.001) in which: age, totChol, sysBP,BMI, heartRate, and glucose are
## the predictor variables, and TenYearCHD is the target variable

## Building the model
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.001).fit(X_train, Y_train)

## Predicting on the test set
ada_preds = ada_md.predict_proba(X_test)[:, 1]

## Using 10% as cutoff value
ada_preds = np.where(ada_preds < 0.1, 0, 1)

## Reporting the recall
print('Recall Score of AdaBoost Classifier:', recall_score(Y_test, ada_preds))

Recall Score of AdaBoost Classifier: 0.9285714285714286


In [18]:
## 5. Using the train data-frame to build an gradient boosting model (with 500 trees, max tree
## depth equal 3, and learning rate equal to 0.001) in which: age, totChol, sysBP,BMI, heartRate, and
## glucose are the predictor variables, and TenYearCHD is the target variable

## Building the model
gb_md = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.001).fit(X_train, Y_train)

## Predicting on the test set
gb_preds = gb_md.predict_proba(X_test)[:, 1]

## Using 10% as cutoff value
gb_preds = np.where(gb_preds < 0.1, 0, 1)

## Reporting the recall
print('Recall Score of Gradient Boosting Classifier:', recall_score(Y_test, gb_preds))

Recall Score of Gradient Boosting Classifier: 1.0


In [ ]:
## 6. Using the results from parts (4) and (5), we would use the Gradient Boosting model to predict TenYearCHD
## because it has the higher recall score of the two consiered models.